In [1]:
import pandas as pd

In [2]:
temp = pd.read_csv('../raw_data/SP_stocks.csv')

In [3]:
temp

,Symbol,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Price to TTM earnings,Price to TTM sales,Price to book value,Action
0,AAPL,Apple Inc,Common stocks,Large cap,Information Technology,"$2,403,239,386,260",0.60%,0,0,0,Analyze
1,MSFT,Microsoft Corp,Common stocks,Large cap,Information Technology,"$1,845,578,460,992",1.02%,0,0,0,Analyze
2,GOOG,Alphabet Inc Class C,Common stocks,Large cap,Communication Services,"$1,189,520,100,000",0.00%,0,0,0,Analyze
3,AMZN,Amazon.Com Inc.,Common stocks,Large cap,Consumer Discretionary,"$960,281,707,587",0.00%,0,0,0,Analyze
4,BRK.B,Berkshire Hathaway Inc. Class B,Common stocks,Large cap,Financials,"$696,358,402,118",0.00%,0,0,0,Analyze
...,...,...,...,...,...,...,...,...,...,...,...
497,LUMN,"CenturyLink, Inc.",Common stocks,Mid cap,Communication Services,"$6,083,347,117",16.45%,0,0,0,Analyze
498,ALK,Alaska Air Group Inc,Common stocks,Mid cap,Industrials,"$5,877,665,089",0.00%,0,0,0,Analyze
499,NWL,Newell Brands Inc,Common stocks,Mid cap,Consumer Discretionary,"$5,451,248,000",6.86%,0,0,0,Analyze
500,VNO,Vornado Realty Trust,REITs,Mid cap,Real Estate,"$4,730,199,427",8.34%,0,0,0,Analyze


In [4]:
from dynamic_portfolio.utils import load_csv

In [5]:
load_csv('AAPL')

,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,reportedEPS,...,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price
0,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,185813.0,0.0399,4.0,0.03,128992.0,168.8,213709.0,45983.0,NaN,NaN
1,2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-03,104.870,112.50,101.690,111.94,0.851857,4783900.0,0.0,1.0,NaN,...,NaN,0.0543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.220001
3,2000-01-04,108.250,110.62,101.190,102.50,0.780019,4574800.0,0.0,1.0,NaN,...,NaN,0.0538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.410004
4,2000-01-05,103.750,110.56,103.000,104.00,0.791434,6949300.0,0.0,1.0,NaN,...,NaN,0.0541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.379997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361,2022-11-22,148.130,150.42,146.925,150.18,150.180000,51804132.0,0.0,1.0,NaN,...,NaN,0.0383,NaN,NaN,NaN,NaN,NaN,NaN,1738.300049,107.220001
8362,2022-11-23,149.450,151.83,149.340,151.07,151.070000,58301395.0,0.0,1.0,NaN,...,NaN,0.0383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8363,2022-11-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8364,2022-11-25,148.305,148.88,147.120,148.11,148.110000,35195860.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
temp = pd.read_csv('/Users/ericvincent/Desktop/gold.csv', sep=';')

In [21]:
temp

,date,gold_price,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,2000-01-03,"288,5",NaN,NaN,NaN,NaN,NaN
1,2000-01-04,"282,2",NaN,NaN,NaN,NaN,NaN
2,2000-01-05,"280,75",NaN,NaN,NaN,NaN,NaN
3,2000-01-06,"280,85",NaN,NaN,NaN,NaN,NaN
4,2000-01-07,"281,5",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5956,2022-11-21,"1737,8081",NaN,NaN,NaN,NaN,NaN
5957,2022-11-22,"1740,24",NaN,NaN,NaN,NaN,NaN
5958,2022-11-23,"1749,28",NaN,NaN,NaN,NaN,NaN
5959,2022-11-24,"1754,96",NaN,NaN,NaN,NaN,NaN


In [22]:
temp = temp.loc[:,['date','gold_price']]

In [23]:
temp

,date,gold_price
0,2000-01-03,"288,5"
1,2000-01-04,"282,2"
2,2000-01-05,"280,75"
3,2000-01-06,"280,85"
4,2000-01-07,"281,5"
...,...,...
5956,2022-11-21,"1737,8081"
5957,2022-11-22,"1740,24"
5958,2022-11-23,"1749,28"
5959,2022-11-24,"1754,96"
